# IMDB MySQL Database Part One
* James Belk
* 3/22/2023

## Imports

In [25]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

## Load Data

In [26]:
# basics_url = 'https://datasets.imdbws.com/title.basics.tsv.gz'
# akas_url = 'https://datasets.imdbws.com/title.akas.tsv.gz'
# ratings_url = 'https://datasets.imdbws.com/title.ratings.tsv.gz'

In [89]:
basics = pd.read_csv('Data/title_basics.csv.gz', low_memory = False)


In [86]:
akas = pd.read_csv('Data/title_akas.csv.gz', low_memory=False)

In [85]:
ratings = pd.read_csv('Data/title_ratings.csv.gz', low_memory=False)

### Save Datasets to 'Data' Folder

In [88]:
basics.to_csv('Data/title_basics.csv.gz',compression='gzip',index=False)

In [83]:
akas.to_csv('Data/title_akas.csv.gz', compression='gzip', index = False)

In [84]:
ratings.to_csv('Data/title_ratings.csv.gz', compression='gzip', index = False)

# Clean Data

## Title Basics

In [87]:
display(basics.head())
display(basics.info())

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892.0,NaN,5,"Animation,Short"
1,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893.0,NaN,1,"Comedy,Short"
2,tt0000006,short,Chinese Opium Den,Chinese Opium Den,0,1894.0,NaN,1,Short
3,tt0000007,short,Corbett and Courtney Before the Kinetograph,Corbett and Courtney Before the Kinetograph,0,1894.0,NaN,1,"Short,Sport"
4,tt0000009,movie,Miss Jerry,Miss Jerry,0,1894.0,NaN,45,Romance


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 714643 entries, 0 to 714642
Data columns (total 9 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   tconst          714643 non-null  object 
 1   titleType       714643 non-null  object 
 2   primaryTitle    714643 non-null  object 
 3   originalTitle   714643 non-null  object 
 4   isAdult         714643 non-null  int64  
 5   startYear       707871 non-null  float64
 6   endYear         17538 non-null   float64
 7   runtimeMinutes  714643 non-null  int64  
 8   genres          714643 non-null  object 
dtypes: float64(2), int64(2), object(5)
memory usage: 49.1+ MB


None

### Replace '\N'

In [31]:
basics = basics.replace({'\\N': np.nan})
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,NaN,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,NaN,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,NaN,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,NaN,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,NaN,1,"Comedy,Short"


### Eliminate Movies That Are Null For 'runtimeMinutes'

In [66]:
runtime_drop = basics.dropna(subset=['runtimeMinutes'], inplace = True)

### Eliminate Movies That Are Null For Genre

In [67]:
genre_null = basics.dropna(subset=['genres'], inplace = True)

### Keep Only 'titleType' Movie

In [68]:
movie_type = basics['titleType']=='Movie'

### Keep 'startYear' 2000-2002

In [77]:
date_range = basics['startYear']==('2000', '2001', '2002')

### Eliminate Movies That Include 'Documentary' in Genre

In [78]:
is_documentary = basics['genres'].str.contains('documentary', case = False)
basics = basics[~is_documentary]

### Keep Only US Movies

In [79]:
keepers = basics['tconst'].isin(akas['titleId'])
keepers

1           True
2          False
3          False
4           True
5           True
           ...  
9710907    False
9710940    False
9710975    False
9710982    False
9710983    False
Name: tconst, Length: 2375360, dtype: bool

In [80]:
basics = basics[keepers]
basics

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,NaN,5,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,NaN,1,"Comedy,Short"
5,tt0000006,short,Chinese Opium Den,Chinese Opium Den,0,1894,NaN,1,Short
6,tt0000007,short,Corbett and Courtney Before the Kinetograph,Corbett and Courtney Before the Kinetograph,0,1894,NaN,1,"Short,Sport"
8,tt0000009,movie,Miss Jerry,Miss Jerry,0,1894,NaN,45,Romance
...,...,...,...,...,...,...,...,...,...
9710677,tt9916214,short,Drown the Clown,Drown the Clown,0,2019,NaN,8,"Drama,Short"
9710697,tt9916254,video,Big Tit Cream Pie 32,Big Tit Cream Pie 32,1,2015,NaN,226,Adult
9710743,tt9916348,video,Ancient World Exposed,Ancient World Exposed,0,2019,NaN,67,History
9710750,tt9916362,movie,Coven,Akelarre,0,2020,NaN,92,"Drama,History"


In [ ]:
basics.info()

## AKAs

In [47]:
display(akas.head())
akas.info()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,1,Карменсіта,UA,\N,imdbDisplay,\N,0
1,tt0000001,2,Carmencita,DE,\N,\N,literal title,0
2,tt0000001,3,Carmencita - spanyol tánc,HU,\N,imdbDisplay,\N,0
3,tt0000001,4,Καρμενσίτα,GR,\N,imdbDisplay,\N,0
4,tt0000001,5,Карменсита,RU,\N,imdbDisplay,\N,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35330391 entries, 0 to 35330390
Data columns (total 8 columns):
 #   Column           Dtype 
---  ------           ----- 
 0   titleId          object
 1   ordering         int64 
 2   title            object
 3   region           object
 4   language         object
 5   types            object
 6   attributes       object
 7   isOriginalTitle  object
dtypes: int64(1), object(7)
memory usage: 2.1+ GB


In [53]:
akas['region'].isin(['US']).any()

True

In [54]:
usa_filter = akas['region']=='US'
akas = akas.loc[usa_filter, :]
akas['region'].value_counts(dropna = False)

US    1424064
Name: region, dtype: int64

In [55]:
akas.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
5,tt0000001,6,Carmencita,US,\N,imdbDisplay,\N,0
14,tt0000002,7,The Clown and His Dogs,US,\N,\N,literal English title,0
33,tt0000005,10,Blacksmith Scene,US,\N,imdbDisplay,\N,0
36,tt0000005,1,Blacksmithing Scene,US,\N,alternative,\N,0
41,tt0000005,6,Blacksmith Scene #1,US,\N,alternative,\N,0


In [56]:
akas = akas.replace({'\\N': np.nan})
akas

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
5,tt0000001,6,Carmencita,US,NaN,imdbDisplay,NaN,0
14,tt0000002,7,The Clown and His Dogs,US,NaN,NaN,literal English title,0
33,tt0000005,10,Blacksmith Scene,US,NaN,imdbDisplay,NaN,0
36,tt0000005,1,Blacksmithing Scene,US,NaN,alternative,NaN,0
41,tt0000005,6,Blacksmith Scene #1,US,NaN,alternative,NaN,0
...,...,...,...,...,...,...,...,...
35329917,tt9916560,1,March of Dimes Presents: Once Upon a Dime,US,NaN,imdbDisplay,NaN,0
35329987,tt9916620,1,The Copeland Case,US,NaN,imdbDisplay,NaN,0
35330076,tt9916702,1,Loving London: The Playground,US,NaN,NaN,NaN,0
35330119,tt9916756,1,Pretty Pretty Black Girl,US,NaN,imdbDisplay,NaN,0


In [ ]:
akas.info()

## Ratings

In [57]:
display(ratings.head())
ratings.info()

,tconst,averageRating,numVotes
0,tt0000001,5.7,1961
1,tt0000002,5.8,263
2,tt0000003,6.5,1799
3,tt0000004,5.6,179
4,tt0000005,6.2,2600


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1292598 entries, 0 to 1292597
Data columns (total 3 columns):
 #   Column         Non-Null Count    Dtype  
---  ------         --------------    -----  
 0   tconst         1292598 non-null  object 
 1   averageRating  1292598 non-null  float64
 2   numVotes       1292598 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 29.6+ MB


In [60]:
usa = ratings['tconst'].isin(akas['titleId'])
usa

0          True
1          True
4          True
5          True
6          True
           ... 
1292563    True
1292564    True
1292571    True
1292572    True
1292576    True
Name: tconst, Length: 492984, dtype: bool

In [64]:
ratings = ratings[usa]
ratings

,tconst,averageRating,numVotes
0,tt0000001,5.7,1961
1,tt0000002,5.8,263
4,tt0000005,6.2,2600
5,tt0000006,5.1,178
6,tt0000007,5.4,817
...,...,...,...
1292563,tt9916200,8.2,224
1292564,tt9916204,8.2,256
1292571,tt9916348,8.3,18
1292572,tt9916362,6.4,5242


In [65]:
ratings = ratings.replace({'\\N': np.nan})
ratings

,tconst,averageRating,numVotes
0,tt0000001,5.7,1961
1,tt0000002,5.8,263
4,tt0000005,6.2,2600
5,tt0000006,5.1,178
6,tt0000007,5.4,817
...,...,...,...
1292563,tt9916200,8.2,224
1292564,tt9916204,8.2,256
1292571,tt9916348,8.3,18
1292572,tt9916362,6.4,5242


In [ ]:
ratings.info()